# Multi-Class Classification with LSTM 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.9

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://securi

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2


update-alternatives: using /usr/bin/python3.9 to provide /usr/bin/python3 (python3) in auto mode


In [ ]:
! python --version

Python 3.9.15


In [ ]:
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [ ]:
#! sudo apt-get install python3-pip

In [ ]:
#Using TensorFlow backend
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')
STOPWORDS = stopwords.words('english')
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import plotly.plotly as py #issue
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/LSTM/train/data.csv') 

In [ ]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KtNPNYlThgy2KyuuJo7xfQ,BtB3hdwKz4q7BVwAOevqsA,C_EtrXTygRX5RTUOKtO6Dg,1,0,0,0,Tried ordering online. When I asked for toppin...,2015-10-17 03:30:18
1,VpgsJiM3zqrhrwJi8bkz2Q,FN7I6FXIzBvTRtl-TKrFtA,uVJkqQ1U9MH7hBorrwGhBg,1,2,0,1,You get what you see--chicken fingers. They al...,2015-08-07 04:59:42
2,HvrQBZSWADUP3H9E4nf8hQ,6Tt7wjCqp4iFKM0zbs1B8Q,unkGQNCeDkAUOaZAg8HC5w,1,0,0,0,"The service and bathroom was horrible, The soa...",2018-05-08 23:59:44
3,cCd-g7FCLpBBliWwBlFO9w,9tNl6Dqj4_HgDGvZS5At3A,P_x9fEz8efemqOIV46S0tg,1,1,0,0,I have been to many TeX Mex places in my life ...,2015-02-02 13:43:31
4,nmdFMMIHKpOpoRHEXG4ALA,eHrs0CRkj3AE_wx4Jb38hQ,V8H6z5ulGJEkaFUyRfmLKw,1,0,0,0,Slowest service I have ever received in my lif...,2015-07-28 17:53:54


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/LSTM/train/data.csv')  
shuffled_df = df.sample(frac=1)
shuffled_df.to_csv("shuffled_data.csv", index=False)

In [ ]:
df1 = pd.read_csv('shuffled_data.csv')  

In [ ]:
df1.head() 

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,nzqTg1zGfTxVzQPau2GMjw,Qj5GosPPvesoCDWMqPhK-A,mMPvgBgqON8pia_4AgGTUw,1,0,0,0,We have always enjoyed this restaurant and the...,2016-01-31 14:49:59
1,tn9uGwo6fpk3Oc8zQxjFag,1cBN9ZQXmvav-W8GSga3zg,WHFYn-b52ApOFymcKuGoFw,2,0,0,0,"Not terrible, just not good. Steak was pretty...",2017-12-18 21:07:12
2,5PTynmN7PGAMgsGtI-iZDA,nlQGyti20c07rpTSYYoNZQ,JOvNunKJlWhwJRwLmUR9ZA,4,0,0,0,I've often passed this swanky little place in ...,2013-06-24 18:52:16
3,vNmQpeClz3VAV9CpRl7B4A,AVvqUnHDIP0Ak4uB-TmCeQ,PEs5kilsGdB31jSt84ItuA,1,3,0,0,I just got back from Chucks and had to login t...,2018-04-04 17:12:22
4,opYup-fparluAB5JmFFydg,Ss8qDbb9qmIT5YhSybg1SQ,OHzX-ZD9qyoeoxR8Z0dlIA,5,0,0,0,There are few truly great lunch places in Old ...,2012-09-11 02:59:39


In [ ]:
drop_columns = ['review_id','user_id','business_id','useful','funny','cool','date']
df1 = df1.drop(drop_columns, axis=1)

In [ ]:
df1.head()

,stars,text
0,1,We have always enjoyed this restaurant and the...
1,2,"Not terrible, just not good. Steak was pretty..."
2,4,I've often passed this swanky little place in ...
3,1,I just got back from Chucks and had to login t...
4,5,There are few truly great lunch places in Old ...


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   stars   10000 non-null  int64 
 1   text    10000 non-null  object
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [ ]:
df1.stars.value_counts()

1    2000
2    2000
4    2000
5    2000
3    2000
Name: stars, dtype: int64

In [ ]:
def print_plot(index):
    example = df1[df1.index == index][['text', 'stars']].values[0]
    if len(example) > 0:
        print(example[0])
        print('stars:', example[1])

In [ ]:
print_plot(10)

This hotel is very nice, but disappointing for a conference venue. All of the shops close early in the evening...and don't expect to have a late dinner or cocktails, the restaurants close early as well. We had a party of 20 and were asked to leave the Finley Irish Pub because they wanted to close @ 10:00pm.
stars: 1


In [ ]:
df1 = df1.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#   text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df1['text'] = df1['text'].apply(clean_text)

In [ ]:
df1['text'] = df1['text'].str.replace('\d+', '')

<ipython-input-20-edee142d5133>:1: FutureWarning:

The default value of regex will change from True to False in a future version.



In [ ]:
print_plot(10)

hotel nice disappointing conference venue shops close early eveningand dont epect late dinner cocktails restaurants close early well party  asked leave finley irish pub wanted close pm
stars: 1


In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 30000
# Max number of words in each review
MAX_SEQUENCE_LENGTH = 250
# This is fixed
EMBEDDING_DIM = 256

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df1['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 28734 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df1['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (10000, 250)


In [ ]:
Y = pd.get_dummies(df1['stars']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (10000, 5)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9000, 250) (9000, 5)
(1000, 250) (1000, 5)


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(EMBEDDING_DIM, dropout=0.2, recurrent_dropout=0.2)) #100. change embedding dim 
#model.add(Dropout(0.2))#
model.add(Dense(5, activation='softmax'))
#model.add(Dropout(0.2))#
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',f1_m])
print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 256)          7680000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 256)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 5)                 1285      
                                                                 
Total params: 8,206,597
Trainable params: 8,206,597
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 5
batch_size = 60

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
135/135 [==============================] - 80s 566ms/step - loss: 1.3905 - accuracy: 0.3714 - f1_m: 0.1391 - val_loss: 1.2037 - val_accuracy: 0.4356 - val_f1_m: 0.2563
Epoch 2/5
135/135 [==============================] - 68s 502ms/step - loss: 0.9656 - accuracy: 0.5846 - f1_m: 0.4759 - val_loss: 1.0837 - val_accuracy: 0.5233 - val_f1_m: 0.4862
Epoch 3/5
135/135 [==============================] - 69s 514ms/step - loss: 0.6973 - accuracy: 0.7188 - f1_m: 0.6988 - val_loss: 1.1695 - val_accuracy: 0.5211 - val_f1_m: 0.4973
Epoch 4/5
135/135 [==============================] - 74s 548ms/step - loss: 0.4630 - accuracy: 0.8268 - f1_m: 0.8212 - val_loss: 1.4245 - val_accuracy: 0.4956 - val_f1_m: 0.4955
Epoch 5/5
135/135 [==============================] - 69s 512ms/step - loss: 0.3062 - accuracy: 0.8919 - f1_m: 0.8900 - val_loss: 1.6173 - val_accuracy: 0.4944 - val_f1_m: 0.4917


In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f} \n  F1: {:0.3f}'.format(accr[0],accr[1],accr[2]))
loss, accuracy, precision = model.evaluate(X_test, Y_test, verbose=0)


32/32 [==============================] - 3s 83ms/step - loss: 1.5636 - accuracy: 0.4860 - f1_m: 0.4788
Test set
  Loss: 1.564
  Accuracy: 0.486 
  F1: 0.479


# Results Analysis:

In this model, since we self-train embeddings (didn't use word embedding features) and our dataset is small(only 10K reviews), our model becomes overfit during training. The validation loss increases, and the accuracy becomes really high. However, when we test our model on the test set, the accuracy drops drastically. In the future, to avoid overfitting,  we should use word embedding as input for the LSTM model. In the later LSTM_Wordembedding model, to test out my assumption, I use word embedding to do binary class classification, and the model performs better overall.  
